In [ ]:
import sys
import requests
import pandas as pd
import plotly.graph_objects as go
import streamlit as st

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperar variables del contexto de app.py
estado_seleccionado = locals().get("ESTADO_SELECCIONADO", "Nuevo León")
palette = locals().get("active_palette", ['#1a1a1a', '#889064', '#ff9f18'])
active_font = locals().get("active_font", "sans-serif")

# Asegurar al menos 3 colores para los sectores (reciclando si es necesario)
def get_colors(n, pal):
    return [pal[i % len(pal)] for i in range(n)]

sector_colors = get_colors(3, palette)

# TOKEN INEGI
TOKEN = "460ccba7-40b7-08a2-47dd-7301e6b6fbbc"

# Diccionario adaptado a los nombres que usa tu app.py (nombres cortos)
STATE_CODE = {
    "Aguascalientes": "01", "Baja California": "02", "Baja California Sur": "03", "Campeche": "04",
    "Coahuila": "05", "Colima": "06", "Chiapas": "07", "Chihuahua": "08", "Ciudad de México": "09",
    "Durango": "10", "Guanajuato": "11", "Guerrero": "12", "Hidalgo": "13", "Jalisco": "14",
    "México": "15", "Michoacán": "16", "Morelos": "17", "Nayarit": "18", "Nuevo León": "19",
    "Oaxaca": "20", "Puebla": "21", "Querétaro": "22", "Quintana Roo": "23", "San Luis Potosí": "24",
    "Sinaloa": "25", "Sonora": "26", "Tabasco": "27", "Tamaulipas": "28", "Tlaxcala": "29",
    "Veracruz": "30", "Yucatán": "31", "Zacatecas": "32"
}

INDICATOR_IDS = {
    "Primario":   "481848",
    "Secundario": "481850",
    "Terciario":  "481869"
}

# -----------------------------------------------------------------------------
# 2. FUNCIONES DE DATOS
# -----------------------------------------------------------------------------

def _url(ids, geo):
    ids_str = ",".join(ids)
    return (f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/"
            f"{ids_str}/es/{geo}/false/BISE/2.0/{TOKEN}?type=json")

@st.cache_data(show_spinner=False)
def fetch_series_by_state(state_name: str, timeout=30) -> pd.DataFrame:
    # Normalización simple por si acaso (aunque app.py ya envía limpio)
    if state_name not in STATE_CODE:
        # Intento de búsqueda inversa si falla el nombre exacto
        found = False
        for k, v in STATE_CODE.items():
            if state_name.lower() in k.lower():
                state_name = k
                found = True
                break
        if not found:
            return pd.DataFrame() # Retorno vacío manejado luego

    geo = STATE_CODE[state_name]
    ids = list(INDICATOR_IDS.values())

    try:
        r = requests.get(_url(ids, geo), timeout=timeout)
        r.raise_for_status()
        js = r.json()
    except Exception as e:
        st.error(f"Error de conexión con INEGI: {e}")
        return pd.DataFrame()

    rows = []
    # Validación de estructura de respuesta
    if not isinstance(js, dict) or "Series" not in js:
        return pd.DataFrame()

    for it in js.get("Series", []):
        ind_id = str(it.get("INDICADOR",""))
        title  = it.get("TITULO","")
        unit   = it.get("UNIDAD_MEDIDA","")

        # Obtener la observación más reciente válida
        observations = it.get("OBSERVATIONS", [])
        if not observations:
            continue

        # Ordenamos o tomamos la última (la API suele dar histórico, nos interesa el último)
        # En BISE 2.0 suelen venir ordenados cronológicamente, tomamos el último
        last_ob = observations[-1]

        rows.append({
            "indicator_id": ind_id,
            "title": title,
            "unit": unit,
            "time_period": last_ob.get("TIME_PERIOD",""),
            "value": float(last_ob.get("OBS_VALUE")) if last_ob.get("OBS_VALUE") not in (None,"") else 0.0,
            "geo": last_ob.get("COBER_GEO","")
        })

    return pd.DataFrame(rows)

def latest_common_year(df: pd.DataFrame) -> str:
    if df.empty: return ""
    last_by_id = df.groupby("indicator_id")["time_period"].max()
    return last_by_id.min()

def format_currency(value: float) -> str:
    return f"${value:,.0f}"

# -----------------------------------------------------------------------------
# 3. LÓGICA DE VISUALIZACIÓN
# -----------------------------------------------------------------------------

def plot_pib_donut(state_name: str):
    df = fetch_series_by_state(state_name)

    if df.empty:
        st.warning(f"No hay datos disponibles para {state_name} o falló la conexión.")
        return

    last_year = latest_common_year(df)
    dfl = df[df["time_period"] == last_year].copy()

    if dfl.empty:
        st.warning("Datos insuficientes para graficar.")
        return

    # Mapeo ID -> Sección
    id_to_section = {v: k for k, v in INDICATOR_IDS.items()}
    dfl["Section"] = dfl["indicator_id"].map(id_to_section)

    # Ordenar secciones: Primario, Secundario, Terciario
    order = ["Primario", "Secundario", "Terciario"]
    dfl = dfl.set_index("Section").reindex(order).reset_index()

    labels = dfl["Section"].tolist()
    values = dfl["value"].tolist()

    # Manejo de valores nulos (convertir NaN a 0)
    values = [v if pd.notna(v) else 0 for v in values]

    total = sum(values)
    percentages = [f"{round((v/total)*100)}%" if total > 0 else "0%" for v in values]

    # Dona
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.6,
        marker=dict(colors=sector_colors), # Usamos la paleta de la app
        text=percentages,
        textinfo='label+text',
        insidetextorientation='radial',
        textfont=dict(family=active_font, size=14)
    )])

    total_fmt = format_currency(total)

    # Ajustamos el layout para Streamlit
    fig.update_layout(
        title={
            'text': f"Indicador trimestral de la actividad económica - {state_name}<br><span style='font-size:14px; color:gray'></span>",
            'x': 0.5,
            'xanchor': 'center',
            'font': dict(family=active_font, size=20)
        },
        annotations=[
            dict(
                text=f"{total_fmt}<br><span style='font-size:12px'>Millones de pesos</span>",
                x=0.5, y=0.5,
                showarrow=False,
                font=dict(size=18, family=active_font, color="#333")
            )
        ],
        showlegend=True,
        legend=dict(orientation="h", y=-0.1, x=0.5, xanchor="center"),
        font=dict(family=active_font),
        margin=dict(t=80, b=40, l=20, r=20),
        height=500
    )

    st.plotly_chart(fig, use_container_width=True)

# -----------------------------------------------------------------------------
# 4. EJECUCIÓN
# -----------------------------------------------------------------------------

if not estado_seleccionado:
    st.info("Selecciona un estado para ver su actividad económica.")
else:
    with st.spinner(f"Consultando datos económicos de {estado_seleccionado}..."):
        plot_pib_donut(estado_seleccionado)